In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import re
import html
from tqdm import tqdm

In [3]:
folder_path = r'C:\Users\DELL\Desktop\Mengge Li\idxFiles'
idxFiles = os.listdir(folder_path)

In [4]:
def generate_url(row):
    return 'https://www.sec.gov/Archives/' + row['Filename']
def custom_filter(row):
    return row['Form Type'] == '10-K'

In [14]:
combined_files = []
result_df =  pd.DataFrame()
for file in idxFiles:
    if file.find('Q')==-1:
        continue
    file_path = os.path.join(folder_path, file)
    print(file)
    if os.path.isfile(file_path):
        #if file_path == r'C:\Users\DELL\Desktop\Mengge Li\idxFiles\master_2011.idx':
        if file_path.find('master_2011')!=-1 or file_path.find('master_2017_Q3.idx')!=-1:
            df = pd.read_csv(file_path, delimiter='|', skiprows=11, header=None, names=['CIK', 'Company Name', 'Form Type', 'Date Filed', 'Filename'], encoding='latin-1')
        else:
            df = pd.read_csv(file_path, delimiter='|', skiprows=11, header=None, names=['CIK', 'Company Name', 'Form Type', 'Date Filed', 'Filename'])
        df = df[df.apply(custom_filter, axis=1)]
        df['URL'] = df.apply(generate_url, axis=1)
        combined_files.append(df)
result_df = pd.concat(combined_files, ignore_index=True)
result_df['Year'] = result_df['Date Filed'].apply(lambda x: pd.to_datetime(x).year)

master_2001_Q1.idx
master_2001_Q2.idx
master_2001_Q3.idx
master_2002_Q1.idx
master_2002_Q2.idx
master_2002_Q3.idx
master_2003_Q1.idx
master_2003_Q2.idx
master_2003_Q3.idx
master_2004_Q1.idx
master_2004_Q2.idx
master_2004_Q3.idx
master_2005_Q1.idx
master_2005_Q2.idx
master_2005_Q3.idx
master_2006_Q1.idx
master_2006_Q2.idx
master_2006_Q3.idx
master_2007_Q1.idx
master_2007_Q2.idx
master_2007_Q3.idx
master_2008_Q1.idx
master_2008_Q2.idx
master_2008_Q3.idx
master_2009_Q1.idx
master_2009_Q2.idx
master_2009_Q3.idx
master_2010_Q1.idx
master_2010_Q2.idx
master_2010_Q3.idx
master_2011_Q1.idx
master_2011_Q2.idx
master_2011_Q3.idx
master_2012_Q1.idx
master_2012_Q2.idx
master_2012_Q3.idx
master_2013_Q1.idx
master_2013_Q2.idx
master_2013_Q3.idx
master_2014_Q1.idx
master_2014_Q2.idx
master_2014_Q3.idx
master_2015_Q1.idx
master_2015_Q2.idx
master_2015_Q3.idx
master_2016_Q1.idx
master_2016_Q2.idx
master_2016_Q3.idx
master_2017_Q1.idx
master_2017_Q2.idx
master_2017_Q3.idx
master_2018_Q1.idx
master_2018_

In [15]:
columns_to_add = [
    "Item 1. Business",
    "Item 1A. Risk Factors",
    "Item 1B. Unresolved Staff Comments",
    "Item 1C. Cybersecurity",
    "Item 2. Properties",
    "Item 3. Legal Proceedings",
    "Item 4. Mine Safety Disclosures",
    "Item 5. Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities",
    "Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations",
    "Item 7A. Quantitative and Qualitative Disclosures About Market Risk"
]

for column in columns_to_add:
    result_df[column] = ""


In [16]:
result_df.to_csv(r'C:\Users\DELL\Desktop\Mengge Li\10-K_remaining.csv', index=False)

In [2]:
sample_df = pd.read_csv(r'C:\Users\DELL\Desktop\Mengge Li\10-K_remaining.csv')

In [3]:
roman_dict = {
        '1': "I",
        '1A': "I A",
        '1B': "I B",
        '1C': "I C",
        '2': "II",
        '3': "III",
        '4': "IV",
        '5': "V",
        '6': "VI",
        '7': "VII",
        '7A': "VII A"
    }

In [4]:
def find_all_indices(file_text, substring):
    try:
        item_no = substring.split()[1].strip('.')
        pattern = r'(?i)(?:^|\n)(I[\s\n]*t[\s\n]*e[\s\n]*m|I[\s\n]*t[\s\n]*e[\s\n]*m[\s\n]*s[\s\n]*)\s*{}\b\s*[.:\\-]*'
        pattern = pattern.format(item_no)
        matches = [match.start() for match in re.finditer(pattern, file_text, re.MULTILINE)]
        if matches==[]:
            roman_no = roman_dict[item_no]
            pattern = r'(?i)(?:^|\n)(I[\s\n]*t[\s\n]*e[\s\n]*m|I[\s\n]*t[\s\n]*e[\s\n]*m[\s\n]*s[\s\n]*)\s*{}\b\s*[.:\\-]*'
            pattern = pattern.format(roman_no)
            matches = [match.start() for match in re.finditer(pattern, file_text, re.MULTILINE)]
    except:
        matches = []
    return matches

def find_item_text(file_text, substring, index):
    item_indices = find_all_indices(file_text, substring)
    if substring=='ITEM 1.' and index!=0:
        item_indices = [i for i in item_indices if i >= index]
    elif substring!='ITEM 1.':
        item_indices = [i for i in item_indices if i >= index]

    if item_indices:
        for start in item_indices:
            item_start_index = start
            startcheck = start
            if file_text[item_start_index]=='\n':
                startcheck +=1
            lines = file_text.splitlines()
            matching_line = None
            current_index = 0
            for i, line in enumerate(lines):
                line_length = len(line)
                if i < len(lines) - 1:
                    line_length += 1  
                if current_index <= startcheck < current_index + line_length:
                    matching_line = line
                    break
                current_index += line_length
            line = matching_line
            if line==None:
                pass
            elif '"' not in line and '“' not in line and '”' not in line and '...' not in line and ',' not in line:
                item_start_index = start
                break
        pattern = re.compile(r'(?i)(?:^|\n)(I[\s\n]*t[\s\n]*e[\s\n]*m|I[\s\n]*t[\s\n]*e[\s\n]*m[\s\n]*s[\s\n]*)\s*[0-9A-C]+\s*[.:\\-]*', re.MULTILINE)
        stop = False
        count=0
        start_index = item_start_index
        while True:
            count+=1
            if stop == True:
                break
            if count>15:
                return '', -1
            match = pattern.search(file_text, pos=start_index+10)
            if match:
                stop_index = match.start()
                item_stop_index = stop_index
                stopcheck = stop_index
                if file_text[item_stop_index]=='\n':
                    stopcheck+=1
                matching_line = None
                current_index = 0
                for i, line in enumerate(lines):
                    line_length = len(line)
                    if i < len(lines) - 1:
                        line_length += 1  
                    if current_index <= stopcheck < current_index + line_length:
                        matching_line = line
                        break
                    current_index += line_length
                line = matching_line
                if line==None:
                    start_index = stop_index
                    continue
                else:
                    try:
                        if '"' not in line and '“' not in line and '”' not in line and '...' not in line and ',' not in line:
                            item_stop_index = stop_index
                            item_text  = file_text[item_start_index:item_stop_index]
                            stop = True
                            return item_text, item_stop_index 
                        else:
                            start_index = stop_index 
                            continue
                    except:
                        start_index = stop_index
                        continue
            else:
                break
        return '', -1
    else:
        return '', -1

def clean_data(text):
    #text = re.sub(r'<.*?>', '', text)
    #text = re.sub(r'-\d+-', '', text)
    text = re.sub(r'\n+', '\n', text)
    text = text.strip()
    return text

In [5]:
items = {
    'ITEM 1.':"Item 1. Business",
    'ITEM 1A.': "Item 1A. Risk Factors",
    'ITEM 1B.' : "Item 1B. Unresolved Staff Comments",
    'ITEM 1C.' : "Item 1C. Cybersecurity",
    'ITEM 2.': "Item 2. Properties",
    'ITEM 3.' : "Item 3. Legal Proceedings",
    'ITEM 4.' : "Item 4. Mine Safety Disclosures",
    'ITEM 5.': "Item 5. Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities",
    'ITEM 7.' : "Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations",
    'ITEM 7A.' : "Item 7A. Quantitative and Qualitative Disclosures About Market Risk"
}

In [6]:
import time
from requests import HTTPError

In [7]:
def extract_info_from_url(row):
    url = row['URL']
    try:
        max_retries = 3
        retry_delay = 3 
        for _ in range(max_retries):
            try:
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
                response = requests.get(url, headers=headers)
                response.raise_for_status()
            except HTTPError as e:
                if e.response.status_code == 503:
                    print(f"Failed to retrieve content. Status code: {e.response.status_code}")
                    time.sleep(retry_delay)  
                    continue
                else:
                    print(f"HTTP error: {e}")
                    break
            except Exception as e:
                print(f"An error occurred: {e}")
                break
        #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        #response = requests.get(url, headers=headers)
        if response.status_code == 200:
            text = response.text 
            #soup = BeautifulSoup(text, 'html.parser')
            #text = soup.get_text()
            text = html.unescape(text)
            text=text.replace('&nbsp;', '')
            text=text.replace('&#151;', '-')
            text=text.replace('&#160;', '-')
            text=text.replace('\xa0', ' ')
            #if url =='https://www.sec.gov/Archives/edgar/data/1046860/0001019892-04-000319.txt' or url =='https://www.#sec.gov/Archives/edgar/data/50601/0001206774-04-001710.txt':
            #    text=text.replace('ITEM I.', 'ITEM 1.')
            #text=text.replace('PART I Item 1. Business:', 'Item 1. Business:') #year 2002 https://www.sec.gov/Archives/edgar/data/79677/0000079677-02-000005.txt
            part_url=['https://www.sec.gov/Archives/edgar/data/23197/0001169232-02-002122.txt','https://www.sec.gov/Archives/edgar/data/70145/0000070145-03-000112.txt' ,'https://www.sec.gov/Archives/edgar/data/70793/0000910647-03-000404.txt','https://www.sec.gov/Archives/edgar/data/1014052/0000096313-03-000216.txt','https://www.sec.gov/Archives/edgar/data/38777/0000038777-03-000693.txt', 'https://www.sec.gov/Archives/edgar/data/1014052/0001015402-04-004698.txt','https://www.sec.gov/Archives/edgar/data/1046050/0000939057-04-000463.txt','https://www.sec.gov/Archives/edgar/data/1283858/0000939057-04-000485.txt','https://www.sec.gov/Archives/edgar/data/717724/0000717724-04-000008.txt','https://www.sec.gov/Archives/edgar/data/1046050/0000939057-05-000360.txt','https://www.sec.gov/Archives/edgar/data/809933/0000809933-05-000019.txt','https://www.sec.gov/Archives/edgar/data/1046050/0000939057-07-000444.txt','https://www.sec.gov/Archives/edgar/data/1305014/0001305014-07-000308.txt','https://www.sec.gov/Archives/edgar/data/1046050/0000939057-08-000478.txt','https://www.sec.gov/Archives/edgar/data/1305014/0001305014-08-000364.txt','https://www.sec.gov/Archives/edgar/data/1046050/0000939057-09-000370.txt','https://www.sec.gov/Archives/edgar/data/1305014/0001305014-09-000097.txt','https://www.sec.gov/Archives/edgar/data/1046050/0000939057-10-000384.txt','https://www.sec.gov/Archives/edgar/data/1305014/0001305014-10-000079.txt']

            text = re.sub(r'<!--[\s\S]*?-->|<[^>]*>', '\n', text)
            emptypattern = re.compile(r'^\s*$', re.MULTILINE)
            text = re.sub(emptypattern, '', text)
            lines = [line.lstrip() for line in text.split('\n')]
            text = '\n'.join(lines)
            flag = False
            table_stop_index = 0
            table_stop_pattern = re.compile(r'Signatures', re.IGNORECASE)
            table_start_index = text.lower().find("table of contents")
            if table_start_index!=-1: 
                start_index = 0
                while True:
                    toc_index = text.lower().find("table of contents", start_index)
                    if toc_index==-1:
                        toc_index = text.lower().find("index to annual report on form 10-k", start_index)
                    if flag==True:
                        break
                    if toc_index != -1:
                        match = table_stop_pattern.search(text, pos=toc_index+1)
                        if match:
                            table_stop_index = match.start()
                            content_after_toc = text[toc_index:table_stop_index]
                            content_lines = content_after_toc.split('\n')
                            found_items = [re.search(r'(?i)Item\s*[0-9A-C]+\s*[.:\\-]*', line) for line in content_lines]
                            if any(found_items):
                                table_start_index = toc_index
                                flag=True
                                break 
                            else:
                                start_index = toc_index + 10
                        else:
                            break
                    else:
                        break
                
                if len(text[table_start_index:table_stop_index])<=3500 and table_stop_index!=0:
                    text = text[:table_start_index] + text[table_stop_index:]
                   
            table_pattern =r'\s*Item\s*[0-9A-C]+\s*[.:\\-]*\s+[A-Za-z0-9\s\'"\-,]+\.*\s*\.{3,}\s*\d+$'
            #text = re.sub(table_pattern, '', text, flags=re.MULTILINE | re.IGNORECASE)
            #text = re.sub(r'<!--[\s\S]*?-->|<[^>]*>', '\n', text)
            #text = re.sub(r'<[^>]*>', '\n', text)
            #text = re.sub(r'<[^>]+>', '\n', text)
            #emptypattern = re.compile(r'^\s*$', re.MULTILINE)
            #text = re.sub(emptypattern, '', text)
            #lines = [line.lstrip() for line in text.split('\n')]
            #text = '\n'.join(lines)
            #pprint(text)
            part_pattern = r'\bPART [I1]\b'
            part_start = []
            for match in re.finditer(part_pattern, text, re.IGNORECASE):
                part_start.append(match.start())
            if len(part_start)>=2:
                if part_start[1]-part_start[0]<=2500 or url in part_url:
                    if text[part_start[0]:part_start[1]].lower().count('item')>10:
                        text = text[part_start[1]:]
            text=text.replace('Item l.', 'Item 1.')
            text=text.replace('ITEM l.', 'Item 1.')
            ctext=clean_data(text)
            correct_df = pd.DataFrame({'Cleaned_Text': [ctext]})
            correct_df.to_csv("correct.csv", index=False)
            
            index=0
            index_save=0
            for item in items:
                item_text, index= find_item_text(text, item, index_save)
                if index!=-1:
                    index_save = index
                if item_text!= None:
                    item_text = clean_data(item_text)
                    item1count= 0 
                    if len(item_text)<500 and item=='ITEM 1.' and not re.search(r'(?i)\b(omitted|not applicable)\b[.,;!?]*', item_text):
                        while True:
                            item1count+=1
                            if item1count>7:
                                break
                            if len(item_text)<500 and item=='ITEM 1.' and not re.search(r'(?i)\b(omitted|not applicable)\b[.,;!?]*', item_text):
                                item_text, index= find_item_text(text, item, index_save)
                                row[items[item]] = item_text
                                if index!=-1:
                                    index_save = index
                            else:
                                break
                    else:
                        row[items[item]] = item_text
                else:
                    row[items[item]] = ' '
        
        else:
            print(f"Failed to retrieve content. Status code: {response.status_code}")
    except Exception as e:
        print(url, e)
    return row

# result_df = result_df.apply(extract_info_from_url, axis=1) /Archives/edgar/data/813781/0001193125-04-111307.txt
tqdm.pandas()
filtered_df = sample_df[sample_df['Year'] == 2015]
filtered_df= filtered_df.progress_apply(extract_info_from_url, axis=1)

100%|██████████| 7428/7428 [12:54:07<00:00,  6.25s/it]   


In [ ]:
filtered_df.to_csv("10-K_2015_Q.csv", index=False) 
filtered_df.to_excel("10-K_2015_Q.xlsx", index=False)

In [2]:
final_df = pd.read_csv(r'C:\Users\DELL\Desktop\Mengge Li\10-K.csv')

In [28]:
#unique_CIK = final_df['CIK'].dropna().unique()
final_df['CIK'].fillna(0, inplace=True)
final_df['CIK'] = final_df['CIK'].astype(int)
unique_CIK = final_df['CIK'].unique()
print(len(unique_CIK))

32077


In [16]:
input = pd.read_excel(r'C:\Users\DELL\Desktop\Mengge Li\Compustat_2001-2022_Firms.xlsx')

In [19]:
input['cik'].fillna(0, inplace=True)
input['cik'] = input['cik'].astype(int)
cik = input['cik'].unique()
print(len(cik))

19264


In [29]:
present=[]
absent=[]
for c in cik:
    if c in unique_CIK:
        present.append(c)
    else:
        absent.append(c)


In [30]:
print(len(present))

13305


In [31]:
print(len(absent))

5959


manual copy paste
https://www.sec.gov/Archives/edgar/data/356309/0000356309-22-000098.txt
https://www.sec.gov/Archives/edgar/data/732026/0001654954-21-010701.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-21-000095.txt
https://www.sec.gov/Archives/edgar/data/1750398/0001477932-20-007321.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-20-000098.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-19-000133.txt
https://www.sec.gov/Archives/edgar/data/1581545/0001683168-18-003066.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-18-000139.txt
https://www.sec.gov/Archives/edgar/data/1335493/0001213900-17-010559.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-17-000185.txt
https://www.sec.gov/Archives/edgar/data/1335493/0001213900-16-017493.txt
https://www.sec.gov/Archives/edgar/data/1631463/0001477932-16-014402.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-16-000283.txt
https://www.sec.gov/Archives/edgar/data/1335493/0001013762-15-000997.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-15-000097.txt
https://www.sec.gov/Archives/edgar/data/1631463/0001477932-15-007800.txt
https://www.sec.gov/Archives/edgar/data/1497074/0001144204-14-075021.txt
https://www.sec.gov/Archives/edgar/data/1591565/0001557240-14-000803.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-14-000113.txt
https://www.sec.gov/Archives/edgar/data/104207/0000104207-13-000104.txt
https://www.sec.gov/Archives/edgar/data/1261654/0001261654-13-000036.txt
https://www.sec.gov/Archives/edgar/data/1410708/0001372167-13-000139.txt
https://www.sec.gov/Archives/edgar/data/1488366/0001471242-13-000386.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-13-000103.txt
https://www.sec.gov/Archives/edgar/data/1409885/0001127855-12-000570.txt
https://www.sec.gov/Archives/edgar/data/1410708/0001372167-12-000135.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-12-000060.txt
https://www.sec.gov/Archives/edgar/data/1046050/0000939057-11-000379.txt
https://www.sec.gov/Archives/edgar/data/1305014/0001305014-11-000091.txt
https://www.sec.gov/Archives/edgar/data/1410708/0001372167-11-000046.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-11-000033.txt
https://www.sec.gov/Archives/edgar/data/40570/0001140361-11-059150.txt
https://www.sec.gov/Archives/edgar/data/1082431/0001099910-10-000149.txt
https://www.sec.gov/Archives/edgar/data/1354591/0001393905-10-000709.txt
https://www.sec.gov/Archives/edgar/data/1409885/0001103132-10-000038.txt
https://www.sec.gov/Archives/edgar/data/356309/0000356309-10-000025.txt
https://www.sec.gov/Archives/edgar/data/1354591/0001103132-09-000046.txt
https://www.sec.gov/Archives/edgar/data/1379006/0001140361-09-023155.txt
https://www.sec.gov/Archives/edgar/data/1409885/0001103132-09-000035.txt
https://www.sec.gov/Archives/edgar/data/1436624/0001436624-09-000029.txt
https://www.sec.gov/Archives/edgar/data/798371/0001038838-09-000315.txt
https://www.sec.gov/Archives/edgar/data/825322/0001204459-09-001857.txt
https://www.sec.gov/Archives/edgar/data/1069562/0000926236-08-000064.txt
https://www.sec.gov/Archives/edgar/data/1089319/0001121781-08-000630.txt
https://www.sec.gov/Archives/edgar/data/1192494/0001103132-08-000027.txt
https://www.sec.gov/Archives/edgar/data/1345715/0001345715-08-000020.txt
https://www.sec.gov/Archives/edgar/data/1381054/0001056520-08-000743.txt
https://www.sec.gov/Archives/edgar/data/1383006/0001376799-08-000138.txt
https://www.sec.gov/Archives/edgar/data/1391142/0001391142-08-000011.txt
https://www.sec.gov/Archives/edgar/data/14846/0001144204-08-068870.txt
https://www.sec.gov/Archives/edgar/data/9128/0000009128-08-000011.txt
https://www.sec.gov/Archives/edgar/data/1069562/0000926236-07-000104.txt
https://www.sec.gov/Archives/edgar/data/1305014/0001305014-07-000308.txt
https://www.sec.gov/Archives/edgar/data/1336262/0001144204-07-056909.txt
https://www.sec.gov/Archives/edgar/data/1345715/0001345715-07-000012.txt
https://www.sec.gov/Archives/edgar/data/14846/0001144204-07-067454.txt
https://www.sec.gov/Archives/edgar/data/317477/0001144204-07-067692.txt
https://www.sec.gov/Archives/edgar/data/1046050/0000939057-06-000352.txt
https://www.sec.gov/Archives/edgar/data/1336262/0001144204-06-044287.txt
https://www.sec.gov/Archives/edgar/data/1304362/0001104659-05-062610.txt
https://www.sec.gov/Archives/edgar/data/837038/0001042167-05-000418.txt
https://www.sec.gov/Archives/edgar/data/797662/0001102624-04-000176.txt
https://www.sec.gov/Archives/edgar/data/895472/0001068238-04-001434.txt
https://www.sec.gov/Archives/edgar/data/895472/0001068238-04-001457.txt
https://www.sec.gov/Archives/edgar/data/895472/0001068238-04-001460.txt
https://www.sec.gov/Archives/edgar/data/895472/0001068238-04-001463.txt
https://www.sec.gov/Archives/edgar/data/774491/0000774491-03-000004.txt
https://www.sec.gov/Archives/edgar/data/1136796/0000931763-02-003707.txt
https://www.sec.gov/Archives/edgar/data/70793/0000910647-02-000252.txt
https://www.sec.gov/Archives/edgar/data/718130/0000928385-02-003749.txt
https://www.sec.gov/Archives/edgar/data/746598/0000950134-02-012859.txt
https://www.sec.gov/Archives/edgar/data/823277/0000897101-02-000830.txt
https://www.sec.gov/Archives/edgar/data/86902/0000086902-02-000022.txt
https://www.sec.gov/Archives/edgar/data/884144/0000950134-02-013059.txt
https://www.sec.gov/Archives/edgar/data/1014052/0000930661-01-502148.txt
https://www.sec.gov/Archives/edgar/data/70793/0000910647-01-500247.txt
https://www.sec.gov/Archives/edgar/data/718130/0000928385-01-502923.txt
https://www.sec.gov/Archives/edgar/data/809933/0000809933-01-500009.txt
https://www.sec.gov/Archives/edgar/data/823277/0000897101-01-500778.txt
https://www.sec.gov/Archives/edgar/data/874265/0000950116-01-501323.txt
https://www.sec.gov/Archives/edgar/data/884144/0000950134-01-507724.txt
not handled
https://www.sec.gov/Archives/edgar/data/1267919/0001376474-21-000336.txt
https://www.sec.gov/Archives/edgar/data/1267919/0001376474-21-000344.txt
https://www.sec.gov/Archives/edgar/data/1434389/0001434389-20-000007.txt
https://www.sec.gov/Archives/edgar/data/1434389/0001434389-20-000008.txt
https://www.sec.gov/Archives/edgar/data/1750398/0001477932-20-007321.txt (business)
https://www.sec.gov/Archives/edgar/data/1552164/0001552164-17-000010.txt (business)
https://www.sec.gov/Archives/edgar/data/1631463/0001477932-16-014402.txt (business)
https://www.sec.gov/Archives/edgar/data/1631463/0001477932-15-007800.txt (business)
https://www.sec.gov/Archives/edgar/data/1438139/0001264931-14-000513.txt (business)
https://www.sec.gov/Archives/edgar/data/1488366/0001471242-13-000386.txt (7,7A)
https://www.sec.gov/Archives/edgar/data/1409885/0001127855-12-000570.txt (no headlines)
https://www.sec.gov/Archives/edgar/data/1498210/0001477932-12-003829.txt (business)
https://www.sec.gov/Archives/edgar/data/1498210/0001477932-12-004955.txt (business)
https://www.sec.gov/Archives/edgar/data/1521722/0001193125-11-344206.txt
https://www.sec.gov/Archives/edgar/data/1354591/0001393905-10-000709.txt (7,7A)
https://www.sec.gov/Archives/edgar/data/1409885/0001103132-10-000038.txt (7,7A)
https://www.sec.gov/Archives/edgar/data/1354591/0001103132-09-000046.txt (7,7A)
https://www.sec.gov/Archives/edgar/data/1409885/0001103132-09-000035.txt 7,7a
https://www.sec.gov/Archives/edgar/data/763730/0000763730-09-000016.txt business
https://www.sec.gov/Archives/edgar/data/798371/0001038838-09-000315.txt 7,7a
https://www.sec.gov/Archives/edgar/data/1192494/0001103132-08-000027.txt 7, 7a
https://www.sec.gov/Archives/edgar/data/763730/0000763730-08-000015.txt business
https://www.sec.gov/Archives/edgar/data/1023313/0000950135-07-006703.txt business
https://www.sec.gov/Archives/edgar/data/1043473/0001043473-03-000015.txt
no data 
https://www.sec.gov/Archives/edgar/data/1159408/0001104659-12-086315.txt
https://www.sec.gov/Archives/edgar/data/1383088/0001511164-12-000300.txt
https://www.sec.gov/Archives/edgar/data/1383088/0001511164-12-000302.txt (bad data)
https://www.sec.gov/Archives/edgar/data/1459287/0001096906-12-003013.txt (bad data)
https://www.sec.gov/Archives/edgar/data/1386936/0001165527-11-001101.txt
https://www.sec.gov/Archives/edgar/data/1402457/0001131754-10-000112.txt
https://www.sec.gov/Archives/edgar/data/1112682/0001112682-09-000026.txt
https://www.sec.gov/Archives/edgar/data/1305014/0001305014-09-000097.txt
https://www.sec.gov/Archives/edgar/data/1442853/0001096350-09-000120.txt
https://www.sec.gov/Archives/edgar/data/1450088/0001477932-09-000035.txt
https://www.sec.gov/Archives/edgar/data/1372649/0000950123-08-016610.txt
https://www.sec.gov/Archives/edgar/data/1417754/0001144204-08-057535.txt
https://www.sec.gov/Archives/edgar/data/1437074/0001193125-08-258175.txt
https://www.sec.gov/Archives/edgar/data/1285828/0001144204-07-051965.txt
https://www.sec.gov/Archives/edgar/data/1374389/0000902561-07-000200.txt
https://www.sec.gov/Archives/edgar/data/1376666/0001162318-07-000871.txt
https://www.sec.gov/Archives/edgar/data/1391087/0000902561-07-000199.txt
https://www.sec.gov/Archives/edgar/data/1393955/0001162318-07-000872.txt
https://www.sec.gov/Archives/edgar/data/1399241/0001193125-07-270107.txt
https://www.sec.gov/Archives/edgar/data/1285828/0001144204-06-042256.txt
https://www.sec.gov/Archives/edgar/data/1374389/0000902561-06-000241.txt
https://www.sec.gov/Archives/edgar/data/103501/0001108017-05-000792.txt
https://www.sec.gov/Archives/edgar/data/1035423/0001104659-05-056208.txt
https://www.sec.gov/Archives/edgar/data/1066107/0001066107-05-000272.txt
https://www.sec.gov/Archives/edgar/data/1326807/0000950133-05-004904.txt
https://www.sec.gov/Archives/edgar/data/1326973/0000950134-05-023807.txt
https://www.sec.gov/Archives/edgar/data/1082735/9999999997-04-048338.txt
https://www.sec.gov/Archives/edgar/data/1094656/0001094656-04-000026.txt
https://www.sec.gov/Archives/edgar/data/740727/0000740727-02-000002.txt
https://www.sec.gov/Archives/edgar/data/922023/0000922023-02-000042.txt
https://www.sec.gov/Archives/edgar/data/1130764/0001021408-01-511732.txt

In [1]:
csv_folder = r'C:\Users\DELL\Desktop\Mengge Li'
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]
combined_data = pd.DataFrame()
for csv_file in csv_files:
    file_path = os.path.join(csv_folder, csv_file)
    df = pd.read_csv(file_path)
    combined_data = pd.concat([combined_data, df], ignore_index=True)
combined_data.to_csv('final_output.csv', index=False)


In [11]:
json_data = filtered_df.to_json(orient='records')
json_file_name = '10-K_2001.json'
with open(json_file_name, 'w') as json_file:
    json_file.write(json_data)

In [3]:
final_output = pd.read_csv('final_output.csv')

In [ ]:
#CIK	Company Name	Form Type	Date Filed	Filename	URL	Year

In [4]:
unique_CIK = final_output['CIK'].unique()
print(len(unique_CIK))

3215


In [6]:
company_name = final_output['Company Name'].unique()
print(len(company_name))

3855


In [11]:
filename = final_output['Filename'].unique()
print(len(filename))

13562


In [13]:
null_count = final_output['Company Name'].isnull().sum()
print(null_count)

0


In [56]:
#result_df.to_csv("10K_sampleOutput.csv", index=False)
#result_df.to_excel("10K_sampleOutput.xlsx", index=False)  

In [3]:
with open('0000950129-01-504660.html', 'r') as file:
    html_content = file.read()
pprint(html_content)

('-----BEGIN PRIVACY-ENHANCED MESSAGE-----\n'
 'Proc-Type: 2001,MIC-CLEAR\n'
 'Originator-Name: webmaster@www.sec.gov\n'
 'Originator-Key-Asymmetric:\n'
 ' MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n'
 ' TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\n'
 'MIC-Info: RSA-MD5,RSA,\n'
 ' UsrqNkGICet8hYDx0BQIR3+3XtAqs5BWZNLj0rL86hISbeEvbIZYfSKFVq35P7GS\n'
 ' gKUWyYJckGyaa14BVjmNZw==\n'
 '\n'
 '<SEC-DOCUMENT>0000950129-01-504660.txt : 20020413\n'
 '<SEC-HEADER>0000950129-01-504660.hdr.sgml : 20020413\n'
 'ACCESSION NUMBER:\t\t0000950129-01-504660\n'
 'CONFORMED SUBMISSION TYPE:\t10-K\n'
 'PUBLIC DOCUMENT COUNT:\t\t18\n'
 'CONFORMED PERIOD OF REPORT:\t20010930\n'
 'FILED AS OF DATE:\t\t20011226\n'
 '\n'
 'FILER:\n'
 '\n'
 '\tCOMPANY DATA:\t\n'
 '\t\tCOMPANY CONFORMED NAME:\t\t\tSTERLING CHEMICALS HOLDINGS INC /TX/\n'
 '\t\tCENTRAL INDEX KEY:\t\t\t0000795662\n'
 '\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tINDUSTRIAL ORGANIC CHEMICALS '
 '[2860]\n'
 '\t\tIRS NUMBE

In [2]:
input = pd.read_excel(r'C:\Users\DELL\Desktop\Mengge Li\Compustat_2001-2022_Firms.xlsx')
print(input.head(5))

    gvkey  fyear    tic      cusip                          conm  cik   sic
0  132602   2001  OONEF  67088Q106  01 COMMUNIQUE LABORATORY INC  NaN  7372
1  132602   2002  OONEF  67088Q106  01 COMMUNIQUE LABORATORY INC  NaN  7372
2  132602   2003  OONEF  67088Q106  01 COMMUNIQUE LABORATORY INC  NaN  7372
3  132602   2004  OONEF  67088Q106  01 COMMUNIQUE LABORATORY INC  NaN  7372
4  132602   2005  OONEF  67088Q106  01 COMMUNIQUE LABORATORY INC  NaN  7372


In [10]:
input['cik'] = input['cik'].astype(str)
cik = input['cik'].dropna().unique()
print(len(cik))

19264
